# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x

In [ ]:
!nvidia-smi -L

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg3"):
    %cd "/content/drive/My Drive/colab-sg3/stylegan3"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/NVlabs/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets

In [ ]:
%cd "/content/drive/My Drive/colab-sg3/stylegan3"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

In [ ]:
!pip install Ninja opensimplex

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/pokemons512.zip"

!unzip {zip_path} -d /content/

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
!python dataset_tool.py --help

In [ ]:


#update this to the path to your image folder
#dataset_path = "/content/pokemons512"
#give your dataset a name
#dataset_name = "pokemons"



#testset = datasets.ImageFolder(root="/content/s", transform=transforms.ToTensor())
#tfs = transforms.Compose([
#            transforms.Resize(img_size),
#            transforms.ToTensor(),
#            normalize,
#        ])

#you don't need to edit anything here
!python dataset_tool.py --source=/content/pokemons512 --dest=/content/drive/MyDrive/colab-sg3/stylegan3/datasets/pokemons.zip
#!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help for help

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "pokemons.zip"
#how often should the model generate samples and a .pkl file
snapshot_count = 50
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None

#resume

!python train.py --outdir=~/training --cfg=stylegan3-r --data=/content/drive/MyDrive/colab-sg3/stylegan3/datasets/pokemons.zip \
    --gpus=1 --batch=32 --batch-gpu=4 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
    --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl

#don't edit this unless you know what you're doing :)
#!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --augpipe="bg"

# Generate Images

##Random Seeds

In [ ]:
pip install opensimplex

In [ ]:
snapshot="/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00020-sex-11gb-gpu-bg-resumecustom/network-snapshot-000008.pkl"

!python generate.py generate-images --network={snapshot} --seeds=1-1000 --trunc=1.3

##Growing

In [ ]:
import glob
model_names = glob.glob("results/00001-a-mirror-mirrory-11gb-gpu-bg/*.pkl") 

In [ ]:
for model in model_names:
  !python generate.py generate-images \
  --network /content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/{model}\
   --seeds=69 --trunc 0.8 --outdir ./results/growing_a/{model}

##Latent Walk

In [ ]:
snapshot="/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00020-sex-11gb-gpu-bg-resumecustom/network-snapshot-000008.pkl"
seednumbers = "964,977,911,936,833,744,774,790,467,964"
framenumber = "1000"

!python generate.py generate-latent-walk --network={snapshot} --seeds={seednumbers} --frames {framenumber} --trunc=1.3
# !python generate.py generate-latent-walk --network=/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00006-W-11gb-gpu-bg-resumecustom/network-snapshot-000028.pkl --seeds=15,21,31,38,58,69,217,234,279,288,15 --frames 300 --trunc=0.7